In [1]:
from gensim.test.utils import datapath
from gensim import utils
import gensim.models
import tempfile
import time

In [2]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-download-auto-examples-tutorials-run-word2vec-py
def split(word):
    return [char for char in word]

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self):
        self.start = time.time()
        self.count = 0


    def __iter__(self):
        corpus_path = "only_ch.txt"
        for line in open(corpus_path, encoding='utf-8'):
            # one document per line, tokens not separated
            self.count += 1
            if self.count % 50000 == 0:
                end = time.time()
                print(self.count)
                print((end - self.start)/60) # in mins 
            #a ="开 发了 即可 我 能 放"
            #print(utils.simple_preprocess(a))
            #yield utils.simple_preprocess(line)
            yield split(line)

In [3]:
sentences = MyCorpus()

In [4]:
start = time.time()
model = gensim.models.Word2Vec(
    sentences=sentences, 
    vector_size=500, 
    window=5, 
    min_count=1, 
    workers=4, 
    sg=0)
model.save("pre_trained_char_5000000_.txt")

50000
0.23362375497817994
100000
0.39429256518681843
150000
0.5398687243461608
200000
0.672812032699585
250000
0.7983842571576436
300000
0.9286679148674011
350000
1.0541789054870605
400000
1.1797093709309896
450000
2.7588436325391132
500000
4.812607526779175
550000
6.458977099259695
600000
8.015711534023286
650000
9.363691822687786
700000
10.811983398596446
750000
12.167520773410796
800000
13.575595613320669
850000
14.867697203159333
900000
17.38626498778661
950000
19.087218177318572
1000000
20.607272815704345
1050000
22.039596708615623
1100000
23.418015225728354
1150000
24.853239953517914
1200000
26.17637492418289
1250000
27.517410242557524
1300000
29.605925778547924
1350000
31.59586559534073
1400000
33.25214918851852
1450000
34.819137291113535
1500000
36.177667951583864
1550000
37.63674489657084
1600000
38.997951126098634
1650000
40.43968543211619
1700000
41.79551227887472
1750000
44.37429131269455
1800000
46.18216851949692
1850000
47.79282341003418
1900000
49.27706692218781
1950000


In [5]:
vec_king = model.wv['一']
vec_king


array([ 0.3963278 ,  1.5865457 ,  0.08271857, -2.762658  ,  0.90899664,
        0.9909689 , -1.0427434 ,  2.451121  ,  2.31003   ,  0.93386036,
        0.1248228 , -3.8423781 , -3.2176251 , -1.2208227 ,  0.8148067 ,
        2.1029966 , -0.61843604, -0.06241135, -0.6782157 ,  0.7852855 ,
       -0.392278  ,  0.12016153,  1.6436627 , -2.4937234 ,  1.6204042 ,
        0.04791727,  0.25082126, -0.43848383,  1.3901484 , -1.8079466 ,
        0.7494227 ,  1.6683373 , -0.49020025,  0.2382814 , -0.2774227 ,
        2.4546432 , -1.9544834 ,  1.561487  , -0.3446904 ,  1.2387425 ,
        1.5567598 , -0.15610969, -1.9810685 , -0.61511   , -4.8695574 ,
       -0.42571324, -2.2513957 , -2.1359532 ,  0.24026969, -4.2104573 ,
        0.80936575,  1.6860231 ,  1.280108  ,  2.7303588 , -0.83340734,
        1.6960759 ,  1.4417585 ,  1.0787718 , -1.5470295 ,  0.41328448,
        0.33641374,  1.51124   ,  1.4495345 ,  1.0295745 , -0.13284418,
        1.084384  ,  0.49966872,  1.431089  ,  0.0531301 , -3.57

In [6]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

NameError: name 'wv' is not defined

In [ ]:
# save and load 
with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    model.save(temporary_filepath)
    #
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.
    #
    # To load a saved model:
    #
    new_model = gensim.models.Word2Vec.load(temporary_filepath)

In [ ]:
# visual result 
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)